### 1.0 -Load libraries
In our Second Hand Car Price Prediction project we used ... ... ...

In [2]:
import sys
print(sys.executable)
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # library for visualization
import seaborn as sns # library for visualization
import warnings
from tqdm import tqdm
import time
import os
import csv
import requests

warnings.filterwarnings('ignore')
# pd.set_option('display.max_columns', None)
sns.set_palette('Spectral')
sns.set_context('notebook', font_scale=1)
sns.set_style('whitegrid')

# set pandas 3 digits decimal
pd.set_option('display.float_format', '{:.3f}'.format)

/Users/w/anaconda3/bin/python


<h4> 1.1 Load the available data </h4>

In [3]:
pathc = "raw_data/car_data.csv" #dataset 1
df=pd.read_csv(pathc,index_col=0)
car=df.copy()
print(f'There are {car.shape[0]} rows and {car.shape[1]} columns')
car.head(5)

There are 115762 rows and 6 columns


,Model,Year,Status,Mileage,Price,MSRP
0,2022 Acura TLX A-Spec,2022,New,Not available,"$49,445","MSRP $49,445"
1,2023 Acura RDX A-Spec,2023,New,Not available,"$50,895",Not specified
2,2023 Acura TLX Type S,2023,New,Not available,"$57,745",Not specified
3,2023 Acura TLX Type S,2023,New,Not available,"$57,545",Not specified
4,2019 Acura MDX Sport Hybrid 3.0L w/Technology ...,2019,Used,"32,675 mi.","$40,990",$600 price drop


In [4]:
data_path='raw_data/used_cars_data.csv'  #dataset 2
df=pd.read_csv(data_path,index_col=0)
cars=df.copy()
print(f'There are {cars.shape[0]} rows and {cars.shape[1]} columns')
cars.head(5)

There are 7253 rows and 13 columns


,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
S.No.,,,,,,,,,,,,,
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.000,NaN,1.750
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.000,NaN,12.500
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.000,8.61 Lakh,4.500
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.000,NaN,6.000
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.000,NaN,17.740


#### 1.2 Scrape data from internet and append the data to available data

This is done to increase amount of data and to increase coherence.

### 2.0 Data Cleaning and preperation

All the collected data needs to have same attributes. And empty ones should be filled/deleted.